In [11]:
from collections import deque, OrderedDict, defaultdict
import bisect

class FIFOCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = set()
        self.queue = deque()

    def process_request(self, key):
        if key in self.cache:
            return True  # Hit
        # Miss
        if len(self.cache) >= self.capacity:
            evict_key = self.queue.popleft()
            self.cache.remove(evict_key)
        self.cache.add(key)
        self.queue.append(key)
        return False  # Miss

class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()

    def process_request(self, key):
        if key in self.cache:
            self.cache.move_to_end(key)
            return True  # Hit
        # Miss
        if len(self.cache) >= self.capacity:
            self.cache.popitem(last=False)
        self.cache[key] = True
        return False  # Miss

class LFUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.min_freq = 0
        self.key_freq = defaultdict(int)
        self.freq_keys = defaultdict(OrderedDict)

    def process_request(self, key):
        if key in self.key_freq:
            # Update frequency
            freq = self.key_freq[key]
            self.freq_keys[freq].pop(key)
            if not self.freq_keys[freq]:
                del self.freq_keys[freq]
                if self.min_freq == freq:
                    self.min_freq += 1
            new_freq = freq + 1
            self.key_freq[key] = new_freq
            self.freq_keys[new_freq][key] = None
            return True  # Hit
        # Miss
        if len(self.key_freq) >= self.capacity:
            # Evict the least frequently used
            evict_key, _ = self.freq_keys[self.min_freq].popitem(last=False)
            del self.key_freq[evict_key]
            if not self.freq_keys[self.min_freq]:
                del self.freq_keys[self.min_freq]
        self.key_freq[key] = 1
        self.freq_keys[1][key] = None
        self.min_freq = 1
        return False  # Miss

class OptimalCache:
    def __init__(self, capacity, trace):
        self.capacity = capacity
        self.cache = set()
        self.trace = trace
        self.key_positions = defaultdict(list)
        for idx, key in enumerate(trace):
            self.key_positions[key].append(idx)

    def get_next_use(self, key, current_pos):
        positions = self.key_positions.get(key, [])
        idx = bisect.bisect_right(positions, current_pos)
        return positions[idx] if idx < len(positions) else float('inf')

    def process_request(self, current_pos):
        key = self.trace[current_pos]
        if key in self.cache:
            return True  # Hit
        # Miss
        if len(self.cache) >= self.capacity:
            furthest = -1
            evict_key = None
            for candidate in self.cache:
                next_use = self.get_next_use(candidate, current_pos)
                if next_use > furthest or (next_use == furthest and candidate < evict_key):
                    furthest = next_use
                    evict_key = candidate
            if evict_key is not None:
                self.cache.remove(evict_key)
        self.cache.add(key)
        return False  # Miss

# Test the caches
trace = [1, 2, 3, 4, 1, 2, 5, 1, 2, 3, 4, 5] *4
capacity = 3

# Cache Simulation with Compact Code

# Define the caches to test
caches = [
    ('FIFO', FIFOCache),
    ('LRU', LRUCache),
    ('LFU', LFUCache),
    ('Optimal', OptimalCache)
]

for name, cache_class in caches:
    if name == 'Optimal':
        # Optimal cache needs trace in constructor and uses index
        cache = cache_class(capacity, trace)
        misses = 0
        for i in range(len(trace)):
            if not cache.process_request(i):
                misses += 1
    else:
        # Other caches only need capacity and use keys
        cache = cache_class(capacity)
        misses = 0
        for key in trace:
            if not cache.process_request(key):
                misses += 1
    print(f"{name} Misses: {misses}, Miss rate: {100*misses/len(trace):.2f} %")

FIFO Misses: 36, Miss rate: 75.00 %
LRU Misses: 40, Miss rate: 83.33 %
LFU Misses: 28, Miss rate: 58.33 %
Optimal Misses: 25, Miss rate: 52.08 %


In [4]:
class TwoWayCounter:
    def __init__(self):
        self.forward = {}  # Maps indexes to counts
        self.backward = {}  # Maps counts to lists of indexes

    def add(self, index, count):
        # If the index already exists, remove the old count from both maps
        if index in self.forward:
            old_count = self.forward[index]
            if old_count in self.backward:
                self.backward[old_count].remove(index)
                if not self.backward[old_count]:
                    del self.backward[old_count]
            # Update the forward map with the new count
            self.forward[index] += count 
        else:
            self.forward[index] = count 

        # Update the backward map
        if self.forward[index] in self.backward:
            self.backward[self.forward[index]].append(index)
        else:
            self.backward[self.forward[index]] = [index]

    def get_indexes(self, count):
        return self.backward.get(count, [])

    def remove(self, index):
        if index in self.forward:
            count = self.forward[index]
            del self.forward[index]
            if count in self.backward:
                self.backward[count].remove(index)
                if not self.backward[count]:
                    del self.backward[count]

# Example usage:
counter = TwoWayCounter()
counter.add(0, 5)
counter.add(1, 3)
counter.add(4, 5)
print(counter.get_indexes(5))  # Output: [0, 4]
counter.add(0, 7)
print(counter.get_indexes(5))  # Output: [4]
print(counter.get_indexes(7))  # Output: []
print(counter.get_indexes(12))  # Output: [0]

[0, 4]
[4]
[]
[0]


In [5]:
13//2

6